In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to run all arguments and not just the last one

In [2]:
# Training a perceptron via scikit-learn
# we will use iris with only 2 features (petal length and petal width)
from sklearn import datasets
import numpy as np
iris = datasets.load_iris()

X = iris.data[:, [2, 3]]
y = iris.target

In [3]:
np.unique(y) # already stored as 0,1,2 which is recommended for many machine learning libraries

array([0, 1, 2])

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=0)

# we will also standardize the features using using StandardScalar from scikit-learn's preprocessing module
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)   # we used the same scaling parameter to standardize the test set so that the 2 are comparable

StandardScaler(copy=True, with_mean=True, with_std=True)

In [4]:
from sklearn.linear_model import Perceptron
ppn = Perceptron(n_iter=40, eta0=0.1, random_state=0)
ppn.fit(X_train_std, y_train)
# here also, we initiate a new perceptron object and train the model using the fit method; random_state for reproducibility
# of the initial shuffling of the training dataset after each epoch

Perceptron(alpha=0.0001, class_weight=None, eta0=0.1, fit_intercept=True,
      n_iter=40, n_jobs=1, penalty=None, random_state=0, shuffle=True,
      verbose=0, warm_start=False)

In [6]:
# after training a model in scikit-learn, we can make predictions using the predict method
y_pred = ppn.predict(X_test_std)
print("Misclassified Samples: %d" % (y_test != y_pred).sum()) # 4/45 samples misclassified

Misclassified Samples: 4


In [7]:
# A large variety of different performance metrics are available via the module metrics eg we can calculate the 
# classification accuracy of a perceptron
from sklearn.metrics import accuracy_score
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred)) # Don't really need the print here

Accuracy: 0.91


In [10]:
# We can use the plot_decision_regions functions we created to plot the classification
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import warnings


def versiontuple(v):
    return tuple(map(int, (v.split("."))))


def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, c=cmap(idx),
                    marker=markers[idx], label=cl)

    # highlight test samples
    if test_idx:
        # plot all samples
        if not versiontuple(np.__version__) >= versiontuple('1.9.0'):
            X_test, y_test = X[list(test_idx), :], y[list(test_idx)]
            warnings.warn('Please update to NumPy 1.9.0 or newer')
        else:
            X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    c='',
                    alpha=1.0,
                    linewidths=1,
                    marker='o',
                    s=55, label='test set')

In [28]:
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))

In [17]:
plot_decision_regions(X=X_combined_std, y=y_combined,
                      classifier=ppn, test_idx=range(105, 150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')

plt.tight_layout()
# plt.savefig('./figures/iris_perceptron_scikit.png', dpi=300)
plt.show()

In [15]:
# Modelling class probabilities using logistic regression
import matplotlib.pyplot as plt
import numpy as np

# To see what a sigmoid function looks like
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

z = np.arange(-7, 7, 0.1)
phi_z = sigmoid(z)

plt.plot(z, phi_z)
plt.axvline(0.0, color='k')
plt.ylim(-0.1, 1.1)
plt.xlabel('z')
plt.ylabel('$\phi (z)$')

# y axis ticks and gridline
plt.yticks([0.0, 0.5, 1.0])
ax = plt.gca()
ax.yaxis.grid(True)

plt.tight_layout()
# plt.savefig('./figures/sigmoid.png', dpi=300)
plt.show()

(-0.1, 1.1)

([<matplotlib.axis.YTick at 0xb2ebb50>,
 <a list of 3 Text yticklabel objects>)

In [14]:
# The scikit-learn version of logistic regression is highly optimized and supports multiclass settings off the shelf
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1000, random_state=0)

lr.fit(X_train_std, y_train)

accuracy_score(y_test, lr.predict(X_test_std)) # Much better than perceptron

In [19]:
plot_decision_regions(X_combined_std, y_combined,
                      classifier=lr, test_idx=range(105, 150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
# plt.savefig('./figures/logistic_regression.png', dpi=300)
plt.show()

In [29]:
# We can predict the class membership probabilities by using predict_proba function
lr.predict_proba(X_test_std[0, :]) # deprecated: will have to use reshape to pass 1d array in the future

C:\Users\309292\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[  2.05743774e-11,   6.31620264e-02,   9.36837974e-01]])

In [32]:
# Tackling overfitting via regularization
# the concept behind regularization is to introduce additional information(bias) to penalize extreme parameter weights
# The most common form of regularization is L2 regularization (also called L2 shrinkage or weight decay) and
# uses a regularization parameter
# To apply regularization, we need to add the regularization term to the cost function that we define for logistic regression
# lambda = 1/C (regularization parameter): decreasing regularization parameter means increasing regularization strength
# We can visualize that by plotting the L2 regularization path for the two weight coefficients
weights, params = [], []
for c in np.arange(-5, 5):
    lr = LogisticRegression(C=10**c, random_state=0)
    lr.fit(X_train_std, y_train)
    weights.append(lr.coef_[1])
    params.append(10**c)

weights = np.array(weights)
plt.plot(params, weights[:, 0],
         label='petal length')
plt.plot(params, weights[:, 1], linestyle='--',
         label='petal width')
plt.ylabel('weight coefficient')
plt.xlabel('C')
plt.legend(loc='upper left')
plt.xscale('log')
# plt.savefig('./figures/regression_path.png', dpi=300)
plt.show()

LogisticRegression(C=1.0000000000000001e-05, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=0,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=0,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

LogisticRegression(C=0.10000000000000001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=0,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [ ]:
# what about the error rate with multiple cost functions
score = []
for c in np.arange(-5, 5):
    lr = LogisticRegression(C=10**c, random_state=0)
    lr.fit(X_train_std, y_train)
    score.append(accuracy_score(y_test, lr.predict(X_test_std)))
    
# np.hstack(np.arange(-5, 5), np.array(score)) failed to stack

np.arange(-5, 5)
np.array(score) 

In [47]:
# Maximum margin classification with support vector machines
# SVM can be considered to be an extension of the perceptron. While in perceptron, we minimized the misclassification errors,
# in SVM, we will aim to maximize the margin
# Margin: the distance between the separating hyperplane and the training samples closest to this hyperplane (called support vectors)
# the idea is that models with large margins have lower generalization errors and are less prone to overfitting than models
# with small margins. The margin is the objective function and we maximize it under the constraint that the samples are 
# classified correctly

In [51]:
# Dealing with non linearly separable cases with slack variables
# Large values of C correspond to large error penalties and smaller to smaller penalties
# C can be used to tune the width of the margin and tune the bias variance trade-off
# Training an SVM model
from sklearn.svm import SVC
svm = SVC(kernel='linear', C=1, random_state=0)

In [54]:
svm.fit(X_train_std, y_train)
accuracy_score(y_test, svm.predict(X_test_std)) # performs pretty similar to Logistic Regression
# in most practical cases, linear logistic regression and linear SVM generate similar results

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

0.97777777777777775

In [11]:
plot_decision_regions(X_combined_std, y_combined,
                      classifier=svm, test_idx=range(105, 150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
# plt.savefig('./figures/support_vector_machine_linear.png', dpi=300)
plt.show()

NameError: name 'X_combined_std' is not defined

In [56]:
# scikit-learn also offers alternate implimentations using SGDClassifier class, which also supports online learning via the 
# partial_fit method
from sklearn.linear_model import SGDClassifier
ppn = SGDClassifier(loss='perceptron')
lr = SGDClassifier(loss='log')
svm = SGDClassifier(loss='hinge')

In [62]:
# Solving nonlinear problems using a kernel SVM
# To see how a nonlinear classification may look
np.random.seed(0)
X_xor = np.random.randn(200, 2)
y_xor = np.logical_xor(X_xor[:, 0] > 0, X_xor[:, 1] > 0)
y_xor = np.where(y_xor, 1, -1)

plt.scatter(X_xor[y_xor == 1, 0],
            X_xor[y_xor == 1, 1],
            c='b', marker='x',
            label='1')
plt.scatter(X_xor[y_xor == -1, 0],
            X_xor[y_xor == -1, 1],
            c='r',
            marker='s',
            label='-1')

plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.legend(loc='best')
plt.tight_layout()
# plt.savefig('./figures/xor.png', dpi=300)
plt.show()

(-3, 3)

(-3, 3)

In [66]:
# The data is not linearly separable. To solve a non linear problem using SVM, we have to transform the training data into 
# a higher dimension feature space via a mapping function. 
svm = SVC(kernel='rbf', random_state=0, gamma=.1, C=10)
svm.fit(X_xor, y_xor)
plot_decision_regions(X_xor, y_xor, classifier=svm)
plt.legend(loc='upper left')
plt.show() # Pretty good for the training set

# Gamma parameter is the cut off parameter for the gaussian sphere. Increasing gamma will increase the area of influence of 
# each training point and lead to a softer decision boundary

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [68]:
# Applying to the flower dataset
svm = SVC(kernel='rbf', random_state=0, gamma=.2, C=1)
svm.fit(X_train_std, y_train)
accuracy_score(y_test, svm.predict(X_test_std)) # Similar accuracy
plot_decision_regions(X_combined_std, y_combined,
                      classifier=svm, test_idx=range(105, 150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
# plt.savefig('./figures/support_vector_machine_rbf_iris_1.png', dpi=300)
plt.show()

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.2, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

0.97777777777777775

In [69]:
# Increase the value of gamma
svm = SVC(kernel='rbf', random_state=0, gamma=100, C=1)
svm.fit(X_train_std, y_train)
accuracy_score(y_test, svm.predict(X_test_std)) # Accuracy down to 80%
plot_decision_regions(X_combined_std, y_combined,
                      classifier=svm, test_idx=range(105, 150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
# plt.savefig('./figures/support_vector_machine_rbf_iris_1.png', dpi=300)
plt.show()

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=100, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

0.80000000000000004

In [5]:
# Decision Tree
# The splits are based on Information gain which can be derived from 3 methods(gini index, entropy or classification error)
import matplotlib.pyplot as plt
import numpy as np
def gini(p):
    return (p)*(1 - (p)) + (1 - p)*(1 - (1 - p))
def entropy(p):
    return - p*np.log2(p) - (1 - p)*np.log2((1 - p))
def error(p):
    return 1 - np.max([p, 1 - p])

x = np.arange(0.0, 1.0, 0.01)
ent = [entropy(p) if p != 0 else None for p in x]
sc_ent = [e*.5 if e else None for e in ent]
err = [error(i) for i in x]
fig = plt.figure()
ax = plt.subplot(111)

for i, lab, ls, c, in zip([ent, sc_ent, gini(x), err],
                         ['Entropy', 'Entropy_scaled', 'Gini Impurity', 'Misclassification error'],
                         ['-', '-', '--', '-.'],
                         ['black', 'lightgray', 'red', 'green', 'cyan']):
    line = ax.plot(x, i, label=lab, linestyle=ls, lw=2, color=c)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3, fancybox=True, shadow=False)
ax.axhline(y=0.5, linewidth=1, color='k', linestyle='--')
ax.axhline(y=1.0, linewidth=1, color='k', linestyle='--')
plt.ylim([0, 1.1])
plt.xlabel('p(i=1)')
plt.ylabel('Impurity Index')
plt.tight_layout()
#plt.savefig('./figures/impurity.png', dpi=300, bbox_inches='tight')
plt.show()

(0, 1.1)

In [6]:
# Building a decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
tree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [12]:
accuracy_score(y_test, tree.predict(X_test)) # doing just as well as the other methods
X_combined = np.vstack((X_train, X_test))
y_combined = np.hstack((y_train, y_test))
plot_decision_regions(X_combined, y_combined, classifier=tree, test_idx=range(105, 150))
plt.xlabel('petal length [cm]')
plt.ylabel('petal width [cm]')
plt.legend(loc='upper left')
plt.tight_layout()
# plt.savefig('./figures/decision_tree_decision.png', dpi=300)
plt.show()

0.97777777777777775

In [13]:
from sklearn.tree import export_graphviz
export_graphviz(tree, out_file="tree.dot", feature_names=["petal length", "petal width"])

In [18]:
# sklearn let's us export the decision tree as a .dot file after training which we can visualize using the Graphviz program
# the file can also be converted into a png 

In [31]:
# Combining weak to strong learners via random forests. 
# one benefit is that we don't need to worry much about hyperparameter tuning as the model is quite robust from noise from 
# individual trees. The only parameter we really need to care about is the number of trees.
# Larger trees lead to better performace at a cost of computation
# Although it is not normally done in practice, hyperparameters of a random forest can be optimized like
# the size of the bootstrap sample; no. of features randomly chosen for each split.
# A good bias variance trade off is generally obtained if the sample size of the bootstrap sample is the same as in the original
# training set
from sklearn.ensemble import RandomForestClassifier

In [37]:
forest = RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=1, n_jobs=2)
forest.fit(X_train, y_train)
accuracy_score(y_test, forest.predict(X_test)) # lower accuracy than basic tree here
plot_decision_regions(X_combined, y_combined, classifier=forest, test_idx=range(105, 150))
plt.xlabel('petal length [cm]')
plt.ylabel('petal width [cm]')
plt.legend(loc='upper left')
plt.tight_layout()
# plt.savefig('./figures/decision_tree_decision.png', dpi=300)
plt.show()

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=2, oob_score=False, random_state=1,
            verbose=0, warm_start=False)

0.9555555555555556

In [39]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski') # the choice for k is very crucial for an over/under fitting balance
knn.fit(X_train_std, y_train)
accuracy_score(y_test, knn.predict(X_test_std)) # 100% on the test set
plot_decision_regions(X_combined_std, y_combined, 
                      classifier=knn, test_idx=range(105, 150))

plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
# plt.savefig('./figures/k_nearest_neighbors.png', dpi=300)
plt.show()

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

1.0

In [ ]:
# The curse of dimensionality
# It is important to mention that KNN is very susceptible to overfitting due to the curse of dimensionality. 
# The curse of dimensionality describes the phenomenon where the feature
# space becomes increasingly sparse for an increasing number
# of dimensions of a fixed-size training dataset. Intuitively, we
# can think of even the closest neighbors being too far away in a
# high-dimensional space to give a good estimate.
# We have discussed the concept of regularization in the section about logistic regression as one way to avoid overfitting. 
# However, in models where regularization is not applicable such as decision trees and KNN, we can use feature selection and 
# dimensionality reduction techniques to help us avoid the curse of dimensionality. This will be discussed in more detail in the next chapter.